In [168]:
import os
from collections import OrderedDict
import pandas as pd
from shapely import affinity
from shapely.geometry import MultiPoint
from scipy.optimize import minimize

def mov(x,y,angle,geom):
    geom_r = affinity.rotate(geom,angle,origin=(0,0))
    geom_t = affinity.translate(geom_r,x,y)
    return geom_t

def calc_dist(geom1,geom2):
    dist = 0
    geom_list = zip(geom1,geom2)
    for item in geom_list:
        dist+=item[0].distance(item[1])
    return dist

def fun1(x0,geom1, geom2):
    x,y,angle = x0
    n_geom = mov(x,y,angle,geom2)
    return calc_dist(geom1,n_geom)

def df_to_od(df01):
    l = len(list(df01.itertuples()))
    ord_01 = OrderedDict()
    l = len(list(df01.itertuples()))
    for i in range(l):
        z = df01.loc[i]['Num']
        k = df01.loc[df01['Num'] == z]
        ord_01[list(k['Num'].iteritems())[0][1]]=(list(k['X'].iteritems())[0][1],list(k['Y'].iteritems())[0][1],list(k['Dist'].iteritems())[0][1])
    return ord_01

def od_to_df(od):
    l = [[k,*od[k]] for k in od.keys()]
    return pd.DataFrame.from_records(l,columns=['Num','X','Y','Dist'])

In [103]:
df01 = pd.read_csv(r'data/chain/103.csv',sep=';', names=['Num','X','Y','Dist'],dtype={'Num':'str'})

In [104]:
f = r'data/chain/104.csv'


In [107]:
od_01 = df_to_od(df01)
od_01

OrderedDict([('597', (-14227.1757, -336.14280000000002, nan)),
             ('596', (-14205.7968, -321.82400000000001, nan)),
             ('598', (-14252.2691, -340.04570000000001, nan)),
             ('599', (-14274.392, -354.625, nan)),
             ('600', (-14299.916300000001, -358.68380000000002, nan)),
             ('601', (-14321.1641, -372.93189999999998, nan)),
             ('602', (-14346.579400000001, -376.98750000000001, nan)),
             ('603', (-14367.7186, -391.1576, nan)),
             ('595', (-14180.5512, -317.86270000000002, nan)),
             ('593', (-14133.8874, -299.58359999999999, nan)),
             ('594', (-14159.2446, -303.60520000000002, nan)),
             ('597a', (-14229.0198, -330.91739999999999, nan))])

In [108]:
df02 = pd.read_csv(f,sep=';', names=['Num','X','Y'],dtype={'Num':'str','X':'float','Y':'float'})

In [109]:
inter = sorted(set(list(zip(*df01['Num'].iteritems()))[1]).intersection(set(list(zip(*df02['Num'].iteritems()))[1])))

In [134]:
df01

,Num,X,Y,Dist
0,597,-14227.1757,-336.1428,NaN
1,596,-14205.7968,-321.8240,NaN
2,598,-14252.2691,-340.0457,NaN
3,599,-14274.3920,-354.6250,NaN
4,600,-14299.9163,-358.6838,NaN
5,601,-14321.1641,-372.9319,NaN
6,602,-14346.5794,-376.9875,NaN
7,603,-14367.7186,-391.1576,NaN
8,595,-14180.5512,-317.8627,NaN
9,593,-14133.8874,-299.5836,NaN


In [135]:
df01_c = df01.loc[df01['Num'].isin(list(df02['Num']))].sort_values(by='Num')
df02_c = df02.loc[df02['Num'].isin(list(df01['Num']))].sort_values(by='Num')
mp01_c = MultiPoint([i[2:4] for i in df01_c.itertuples()])
mp02 = MultiPoint([i[2:4] for i in df02.itertuples()])
mp02_c = MultiPoint([i[2:4] for i in df02_c.itertuples()])
x0 = (0,0,0)
re = minimize(fun1,x0,args=(mp01_c,mp02_c),method='SLSQP')
print(re)
mp02_c_moved =  mov(*re.x,mp02_c)
ab = list(zip(mp01_c,mp02_c_moved))
mp02_c_avg = MultiPoint([((i[0].x+i[1].x)/2,(i[0].y+i[1].y)/2) for i in ab])
mp02_dist = [i[0].distance(i[1]) for i in ab]
mp02_c_avg_dict = {k:v for k,v in zip(df02_c['Num'],list(zip(mp02_c_avg,mp02_dist)))}
mp_moved = mov(*re.x,mp02)

     fun: 0.002081701982871198
     jac: array([   3.7404325 ,    2.57947205, -439.46588063,    0.        ])
 message: 'Optimization terminated successfully.'
    nfev: 424
     nit: 71
    njev: 71
  status: 0
 success: True
       x: array([ -267.26849401,  2441.26995037,     9.99506175])


In [136]:
mp02_mov_dict = {k:v for k,v in zip(df02['Num'],list(zip(mp_moved,['']*len(mp_moved))))}
for k in mp02_c_avg_dict.keys():
    mp02_mov_dict[k] = mp02_c_avg_dict[k]
stag_02 = []
for k in mp02_mov_dict.keys():
    row = [k,mp02_mov_dict[k][0].x,mp02_mov_dict[k][0].y,mp02_mov_dict[k][1]]
    stag_02.append(row)

In [146]:
print(list(df02_c['Num']))

['598', '599', '600', '601', '602', '603']


In [137]:
stag_02

[['608', -14486.659983688924, -431.8412530591454, ''],
 ['601', -14321.163968178495, -372.9318045415919, 0.0003255101670640496],
 ['606', -14440.156190421494, -413.5942615693084, ''],
 ['609', -14508.084910381547, -446.1187272718935, ''],
 ['603', -14367.718615389818, -391.1574132924443, 0.00037468150653505427],
 ['600', -14299.915980970309, -358.68408497349935, 0.0008555462355794323],
 ['599', -14274.391939397636, -354.6248820920889, 0.00026514088271693066],
 ['605', -14414.777335718443, -409.59959567571013, ''],
 ['607', -14461.387890409183, -427.84346971582454, ''],
 ['598', -14252.269005020758, -340.0456787640323, 0.00019464863284705865],
 ['604', -14393.066408591263, -395.1081345441262, ''],
 ['602', -14346.579372287724, -376.98748192244955, 6.61745581286724e-05]]

In [138]:
od_stag = OrderedDict({i[0]:(i[1],i[2],i[3]) for i in stag_02})

In [141]:
od_01

OrderedDict([('597', (-14227.1757, -336.14280000000002, nan)),
             ('596', (-14205.7968, -321.82400000000001, nan)),
             ('598', (-14252.2691, -340.04570000000001, nan)),
             ('599', (-14274.392, -354.625, nan)),
             ('600', (-14299.916300000001, -358.68380000000002, nan)),
             ('601', (-14321.1641, -372.93189999999998, nan)),
             ('602', (-14346.579400000001, -376.98750000000001, nan)),
             ('603', (-14367.7186, -391.1576, nan)),
             ('595', (-14180.5512, -317.86270000000002, nan)),
             ('593', (-14133.8874, -299.58359999999999, nan)),
             ('594', (-14159.2446, -303.60520000000002, nan)),
             ('597a', (-14229.0198, -330.91739999999999, nan))])

In [142]:
od_stag

OrderedDict([('608', (-14486.659983688924, -431.8412530591454, '')),
             ('601',
              (-14321.163968178495,
               -372.9318045415919,
               0.0003255101670640496)),
             ('606', (-14440.156190421494, -413.5942615693084, '')),
             ('609', (-14508.084910381547, -446.1187272718935, '')),
             ('603',
              (-14367.718615389818,
               -391.1574132924443,
               0.00037468150653505427)),
             ('600',
              (-14299.915980970309,
               -358.68408497349935,
               0.0008555462355794323)),
             ('599',
              (-14274.391939397636,
               -354.6248820920889,
               0.00026514088271693066)),
             ('605', (-14414.777335718443, -409.59959567571013, '')),
             ('607', (-14461.387890409183, -427.84346971582454, '')),
             ('598',
              (-14252.269005020758,
               -340.0456787640323,
               0.0001946486328

In [160]:
for k in od_stag.keys():
    od_01[k] = od_stag[k]

In [169]:
dft = od_to_df(od_01)

In [170]:
dft

,Num,X,Y,Dist
0,597,-14227.175700,-336.142800,NaN
1,596,-14205.796800,-321.824000,NaN
2,598,-14252.269005,-340.045679,0.000194649
3,599,-14274.391939,-354.624882,0.000265141
4,600,-14299.915981,-358.684085,0.000855546
5,601,-14321.163968,-372.931805,0.00032551
6,602,-14346.579372,-376.987482,6.61746e-05
7,603,-14367.718615,-391.157413,0.000374682
8,595,-14180.551200,-317.862700,NaN
9,593,-14133.887400,-299.583600,NaN
